## HOUSING RECOMMENDATIONS IN BANGALORE BASED ON FOOD CATEGORY


### This Notebook will provide housing recommendation in Bangalore Neighborhoods based on the Food Category[From Foursquare]

#### Importing Required Libraries

In [26]:
import pandas as pd
import numpy as np
import matplotlib as mlp
import matplotlib.pyplot as plt
import requests
from io import StringIO
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Code to fetch Bangalore Neighbourhood dataset from Cloud storage.
The code is hidden as it contains storage credentials

In [27]:
# The code was removed by Watson Studio for sharing.

,Boroughs,Neighbourhood,longitude,latitude
0,Bangalore East,Arabic College,77.6206,13.0291
1,Bangalore East,Bellandur,77.6760,12.9298
2,Bangalore East,Domlur,77.6359,12.9611
3,Bangalore East,Dr. Shivarama Karanth Nagar,77.6293,13.0681
4,Bangalore East,Fraser Town,77.6164,13.0005


#### Getting Geographical Coordinates of Bangalore from geopy library

In [28]:
address = 'Bangalore, KA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore city are 12.9791198, 77.5912997.


#### Creating Bangalore map with Neighbourhoods using Folium library

In [7]:
map_Bangalore = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_data_1['latitude'],df_data_1['longitude'], df_data_1['Boroughs'], df_data_1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bangalore)  
    
map_Bangalore

#### User credentials for fetching Foursquare API data

In [8]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: QEQJLISAVGB5BQOBGDDOKQWGALH1DPKOCWKM3JSA0SYPCGZX
CLIENT_SECRET:R02DSR2A10ASSKJOIZIJPNILF5JPACNRPQRLUD0Q0PTL24DX


#### Fetching Venues from Foursquare with the radius of 500m from each Neighbourhoods

In [9]:


def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)



In [10]:
venues = getNearbyVenues(df_data_1['Neighbourhood'],df_data_1['latitude'],df_data_1['longitude'],500)
venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arabic College,13.0291,77.6206,Fiat Aadya Service Center,13.030655,77.620240,Auto Workshop
1,Arabic College,13.0291,77.6206,D.K Communications,13.027063,77.622548,Electronics Store
2,Arabic College,13.0291,77.6206,New Krishna Sagar,13.026125,77.622722,Indian Restaurant
3,Arabic College,13.0291,77.6206,Mateen Marketing,13.033118,77.619645,Furniture / Home Store
4,Bellandur,12.9298,77.6760,Kicks On Grass,12.930045,77.679679,Soccer Field
5,Bellandur,12.9298,77.6760,Bangalore Central,12.926006,77.675301,Shopping Mall
6,Bellandur,12.9298,77.6760,Cafe Coffee Day Central 3,12.926107,77.675755,Café
7,Bellandur,12.9298,77.6760,Domino's Pizza,12.926045,77.676107,Pizza Place
8,Bellandur,12.9298,77.6760,McDonald's,12.925893,77.675356,Fast Food Restaurant
9,Bellandur,12.9298,77.6760,Thalassery Restaurant,12.925845,77.675901,Kerala Restaurant


In [9]:
venues.shape

(812, 7)

#### Filtering the Venues to fetch only Indian Restaurants

In [12]:
Indian_Rest_venues = venues.loc[venues['Venue Category'] == 'Indian Restaurant']

In [13]:
Indian_Rest_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Arabic College,13.0291,77.6206,New Krishna Sagar,13.026125,77.622722,Indian Restaurant
15,Bellandur,12.9298,77.6760,Punjab Di Shaan,12.925613,77.676421,Indian Restaurant
16,Bellandur,12.9298,77.6760,Mast Kalandar,12.925604,77.675966,Indian Restaurant
17,Bellandur,12.9298,77.6760,Punjabi Tadka,12.926037,77.676749,Indian Restaurant
18,Bellandur,12.9298,77.6760,Chatoree,12.929034,77.672394,Indian Restaurant
26,Domlur,12.9611,77.6359,Srinidhi Sagar,12.959348,77.638387,Indian Restaurant
27,Domlur,12.9611,77.6359,Kerala Pavilion,12.960354,77.637207,Indian Restaurant
32,Fraser Town,13.0005,77.6164,Chichaba's Taj,13.000344,77.617174,Indian Restaurant
37,Fraser Town,13.0005,77.6164,Richies - Frazer Town,13.000930,77.613150,Indian Restaurant
42,Fraser Town,13.0005,77.6164,Bombay Chowpatty Kulfi,13.000696,77.615289,Indian Restaurant


In [14]:
Indian_Rest_venues.shape

(157, 7)

#### Getting counts of Indian Restaurants in each Neighbourhoods

In [16]:
IndFreq = Indian_Rest_venues.groupby(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude'])['Venue Category'].agg({'Indian Restaurant':'count'})
IndFreq.reset_index()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if __name__ == '__main__':


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Indian Restaurant
0,Arabic College,13.0291,77.6206,1
1,Ashoknagar,12.9377,77.5645,3
2,Basavanagudi,12.9446,77.5741,9
3,Bellandur,12.9298,77.6760,4
4,Bommanahalli,12.9068,77.6311,2
5,Bommasandra Industrial Estate,12.8203,77.6878,1
6,Carmelram,12.9078,77.6937,1
7,Chandapura,12.8016,77.7041,1
8,Dharmaram College,12.9352,77.5684,1
9,Domlur,12.9611,77.6359,2


#### Clustering and mapping the Neighbourhoods with respect to the frequency of Indian Restaurants

In [19]:
clusters = 5
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(IndFreq[['Indian Restaurant']])
kmeans.labels_[0:10]
IndFreq['Cluster ID'] = kmeans.labels_
IndFreq = IndFreq.reset_index()
IndFreq.head(30)

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Indian Restaurant,Cluster ID
0,0,Arabic College,13.0291,77.6206,1,0
1,1,Ashoknagar,12.9377,77.5645,3,4
2,2,Basavanagudi,12.9446,77.5741,9,1
3,3,Bellandur,12.9298,77.6760,4,3
4,4,Bommanahalli,12.9068,77.6311,2,4
5,5,Bommasandra Industrial Estate,12.8203,77.6878,1,0
6,6,Carmelram,12.9078,77.6937,1,0
7,7,Chandapura,12.8016,77.7041,1,0
8,8,Dharmaram College,12.9352,77.5684,1,0
9,9,Domlur,12.9611,77.6359,2,4


In [21]:
latitude = IndFreq['Neighborhood Latitude'][0]
longitude = IndFreq['Neighborhood Longitude'][0]
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster, qty in zip(IndFreq['Neighborhood Latitude'], IndFreq['Neighborhood Longitude'], IndFreq['Neighborhood'], IndFreq['Cluster ID'], IndFreq['Indian Restaurant']):
    label = folium.Popup(str(poi) + ' - ' + str(qty) + " Indian Restaurant", parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    

legend_html =   '''
                <div style="position: fixed; 
                            top: 50px; left: 50px; width: 100px; height: 100px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Cluster 0 &nbsp; <i class="fa fa-circle" style="color:red"></i><br>
                              &nbsp; Cluster 1 &nbsp; <i class="fa fa-circle" style="color:#9900cc"></i><br>
                              &nbsp; Cluster 2 &nbsp; <i class="fa fa-circle" style="color:#66ccff"></i><br>
                              &nbsp; Cluster 3 &nbsp; <i class="fa fa-circle" style="color:#00ffff"></i><br>
                              &nbsp; Cluster 4 &nbsp; <i class="fa fa-circle" style="color:#ffa366"></i>
                </div>
                ''' 

map_clusters.get_root().html.add_child(folium.Element(legend_html))
       
map_clusters

#### Clusters with the neighbourhood counts

In [22]:
for i in np.arange(clusters):
    print ('Cluster {}: {} Neighbourhoods'.format(i,IndFreq['Cluster ID'].value_counts()[i]))



Cluster 0: 12 Neighbourhoods
Cluster 1: 7 Neighbourhoods
Cluster 2: 2 Neighbourhoods
Cluster 3: 7 Neighbourhoods
Cluster 4: 10 Neighbourhoods


#### Count of Indian Restaurants in each clusters

In [17]:
x=IndFreq.groupby(['Cluster ID']).sum()
x[['Indian Restaurant']]

,Indian Restaurant
Cluster ID,
0,12
1,56
2,35
3,30
4,24


#### Since Clusters 1,2,3 are top three clusters of Indian Restaurants,we'll consider the locations near to these clusters

In [24]:
Final_locations=IndFreq.loc[IndFreq['Cluster ID'].isin(['1','2','4'])]
Final_locations.reset_index(drop=True)

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Indian Restaurant,Cluster ID
0,1,Ashoknagar,12.9377,77.5645,3,4
1,2,Basavanagudi,12.9446,77.5741,9,1
2,4,Bommanahalli,12.9068,77.6311,2,4
3,9,Domlur,12.9611,77.6359,2,4
4,10,Fraser Town,13.0005,77.6164,7,1
5,11,Gottigere,12.8564,77.5883,3,4
6,15,J P Nagar,12.9119,77.5868,8,1
7,16,JP Nagar III Phase,12.9139,77.5976,9,1
8,18,Jayanagar,12.9237,77.5860,7,1
9,19,Jayangar III Block,12.9304,77.5835,19,2



#### Started with 57 Neighbourhoods we are now ended up with around 16,which could be recommended for users,
Lets plot them in the map

In [25]:
map_Final = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,neighborhood in zip(Final_locations['Neighborhood Latitude'],Final_locations['Neighborhood Longitude'],Final_locations['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#99ff33',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Final)  
    
map_Final